Hello dis is test

In [14]:
from Inner_Speech_Dataset.Python_Processing.Data_extractions import  Extract_data_from_subject

datatype = "EEG"
root_dir = "../dataset"
subjectNr = 1

data, label = Extract_data_from_subject(root_dir, subjectNr, datatype)

Hello dis is test


In [24]:
print(label[10]) # [39869     3     0     1]  I think it means this:  [unknown     class?     contaminated?     subjectNr]

[106461      0      0      1]


In [17]:
!ls -la .. # ! to write bash commands

total 65236
drwxr-xr-x 13 jonlis jonlis     4096 feb  4 15:15 .
drwxr-xr-x 51 root   root       4096 feb  3 15:08 ..
-rw-rw-r--  1 jonlis jonlis       44 feb  3 15:57 .bash_aliases
-rw-------  1 jonlis jonlis     1830 feb  3 21:54 .bash_history
-rw-r--r--  1 jonlis jonlis      220 apr  4  2018 .bash_logout
-rw-r--r--  1 jonlis jonlis     4254 feb  3 16:00 .bashrc
drwx------  3 jonlis jonlis     4096 jan 31 20:30 .cache
drwxrwxr-x  2 jonlis jonlis     4096 feb  3 16:00 .conda
drwxrwxr-x  3 jonlis jonlis     4096 feb  4 15:20 dataset
drwxrwxr-x  3 jonlis jonlis     4096 feb  3 15:21 Feasibility_Analysis_Inner_Speech
-rw-rw-r--  1 jonlis jonlis       60 feb  3 14:11 .gitconfig
drwx------  3 jonlis jonlis     4096 nov  1 15:18 .gnupg
drwxrwxr-x  5 jonlis jonlis     4096 feb  3 16:27 inner-speech-detection
drwxr-xr-x  5 jonlis jonlis     4096 feb  3 13:58 .ipython
drwx------  5 jonlis jonlis     4096 feb  3 16:09 .local
drwxrwxr-x 16 jonlis jonlis     4096 feb  3 16:00 miniconda3
-rwxrwxr-x